In [8]:
import numpy as np
from tqdm import tqdm
import csv

# BIO_Main

In [9]:
#隐马尔可夫模型
#面向命名实体识别(组织机构)问题
class HMM_BIO_main:
    def __init__(self,A=None,B=None,PI=None):
        #param:
        # (A,B,PI) 状态转移矩阵 发射矩阵 初始状态矩阵
        self.print=True
        
        
        self.N = 3     # 状态集合有多少元素(状态数BIO)
        self.M = 65535 # 观测集合有多少元素(汉字)
        
        #数字化 状态集合Q 观测集合V 
        self.status_dict={ # {'盒子1':0，'盒子2':1，'盒子3':2}
                        'B-ORG': 0,
                        'I-ORG': 1,
                        'O': 2}
        self.status=list(self.status_dict.keys())          
        #self.observe_dict={}# {'红':0，'白':1}
        self.Q=np.arange(0,self.N) #状态集合 [0,1,2]
        self.V=np.arange(0,self.M) #观测集合 [0,1]
        
        #初始化 (A,B,PI)
        self.A=A # 状态转移矩阵
        self.B=B # 发射矩阵
        self.PI=PI # 初始状态概率
        
        
        if self.print:
            print('状态集合status',self.status_dict)
            #print('观测集合observe',self.observe_dict)
            print('状态集合Q',self.Q)
            print('观测集合V',self.V)
            print()
        

    
    #训练模型(A,B,PI)
    #data (2,N)
    def train(self,filename,savedir=None):
        #利用数据data(状态序列+观测序列) 计算参数(A,B,PI) 直接使用统计方法 统计A,B,PI
        
        A=np.zeros((self.N,self.N)) #(N,N)
        B=np.zeros((self.N,self.M)) #(N,M)
        PI=np.zeros(self.N) #(N,)
        
        
        with open(filename,'r',encoding='utf-8') as fr:
            mult=10000000
            cnt=0
            
            line=fr.readline()
            pres=-1
            while line:
        
                mult-=1
                if mult==0:
                    cnt+=1
                    print(f'load {10000000}*{cnt} rows')
                    mult=10000000
                    
                
                
                s,v=line.split(',')
                s=int(s)
                v=int(v[:-1])
                #print(line,s,v)
                #当前为分隔符
                if s==-1 or v>=self.M:
                    pres=-1
                    line=fr.readline()
                    continue
                if pres!=-1:
                    #s 当前状态
                    #pres 上一次状态
                    #v 当前观测

                    A[pres,s]+=1
                    B[s,v]+=1
                    
                #上一个为分隔符
                else:
                    PI[s]+=1
                    B[s,v]+=1
                pres=s
                line=fr.readline()
        
        self.A=A
        self.B=B
        self.PI=PI
        print('参数统计结果(A,PI)')
        print(f'A:{self.A}')
        print(f'PI:{self.PI}')
        
        
        self.normalize_param()
        print('训练参数结果(A,B,PI)')
        print(f'A:{self.A}')
        print(f'B:{self.B}')
        print(f'PI:{self.PI}')
        
        
        self.save_Param(savedir)
        
        
    
    def normalize_param(self):
        epsilon=1e-8
        
        
        self.PI[self.PI == 0] =epsilon  # 防止数据下溢,对数据进行对数归一化
        self.PI = np.log(self.PI) - np.log(np.sum(self.PI))

        self.A[self.A == 0] = epsilon
        self.A = np.log(self.A) - np.log(np.sum(self.A, axis=1, keepdims=True))

        self.B[self.B == 0] = epsilon
        self.B = np.log(self.B) - np.log(np.sum(self.B, axis=1, keepdims=True))
        
        #将(A,B,PI)归一化 (直接归一化)
        #self.A=self.A/np.sum(self.A,axis=1,keepdims=True)
        #self.B=self.B/np.sum(self.B,axis=1,keepdims=True)
        #self.PI=self.PI/np.sum(self.PI)

    def save_Param(self,savedir):
        if savedir==None:
            savedir='param/BIO/main/'
        np.savetxt(f'{savedir}A.csv',self.A,delimiter=',')
        np.savetxt(f'{savedir}B.csv',self.B,delimiter=',')
        np.savetxt(f'{savedir}PI.csv',self.PI,delimiter=',')
        print(f'训练参数已保存在 {savedir}A.csv')
        print()
    
    def load_Param(self,loaddir='param/BIO/main/'):
        self.A=np.genfromtxt(f'{loaddir}A.csv',delimiter=',')
        self.B=np.genfromtxt(f'{loaddir}B.csv',delimiter=',')
        self.PI=np.genfromtxt(f'{loaddir}PI.csv',delimiter=',')
        print(f'训练参数(A,B,PI)已从 {loaddir}A.csv 中读取成功')
        print()
    
    def viterbi_t(self,o):
        #维特比算法(动态规划)
        #已知(A,B,PI) 和一观测序列o 求解其状态序列的概率最大解
        #param:
        # o 观测序列 [0,1,1,1,1,0] shape(n,)
        #return:
        # ret 状态序列 shape(n,)
        #solution:
        # delta计算从前一个的所有状态 到当前状态的概率最大值 psi记录下来概率最大值的前一个状态
        # 从delta的最后取最大值，利用psi向前回溯即可找到最大概率序列
        
        n=o.shape[0]
        
        delta=np.zeros((self.N,n)) #delta[:,i] 为到达该观测序列的 每个状态的最大概率值 shape(状态个数,序列长度)
        psi=np.zeros((self.N,n),dtype=np.int32) #psi[:,i] 为到达该观测序列的 最大概率值的 上一个状态为哪个
        A=self.A
        B=self.B
        PI=self.PI
        
        #* -> + (因为log)
        delta[:,0]=PI+B[:,o[0]]
        #psi[:,0]=np.argmax(delta[:,0])
        for i in range(1,n):
            temp=delta[:,i-1]+A.T
            psi[:,i]=np.argmax(temp,axis=1)
            delta[:,i]=np.max(temp,axis=1)+B[:,o[i]]
        
        ret=np.zeros(n,dtype=np.int32)
        ret[-1]=np.argmax(delta[:,-1])

        for i in range(n-2,-1,-1):
            ret[i]=psi[ret[i+1]][i+1]
        return ret
    
    
    
    def calc_acc(self,label,predict):
        #计算标签预测的准确率
        #param:
        # a 源标签
        # b 目的标签
        return np.mean(np.equal(label,predict))
    
    
    
    def predict(self,s:str):
        #预测一段话的状态序列
        #param
        # s 一段中文
        #print
        # 中文加状态的文字 林B-LOC_|徽I-LOC_|因_O_|
        
        ans=""
        s=list(s)
        o=np.array([ord(ch) for ch in s])
        #打印观测序列
        #print(o)
        statusList=self.viterbi_t(o)
        for i in range(len(s)):
            ans+=s[i]+str(self.status[statusList[i]])+'_|'
        #print(statusList)
        print(ans)
        

# BIO_Method 1

In [21]:
class HMM_BIO_method1:
    def __init__(self):
        self.n_tag = 3  # 表示所有标签个数
        self.n_char = 65535  # 所有字符的Unicode编码个数
        self.epsilon = 1e-100  # 无穷小量
        self.tag2idx = {'B-ORG': 0,
                        'I-ORG': 1,
                        'O': 2}
        self.idx2tag = dict(zip(self.tag2idx.values(), self.tag2idx.keys()))
        self.A = np.zeros((self.n_tag, self.n_tag))  # 转移概率矩阵,shape:7*7
        self.B = np.zeros((self.n_tag, self.n_char))  # 发射概率矩阵,shape:7*字的个数
        self.pi = np.zeros(self.n_tag)  # 初始隐状态概率,shape：4

    def train(self, corpus_path ,savedir=None):
        """
        函数说明： 训练HMM模型, 得到模型参数pi,A,B

        Parameter：
        ----------
            corpus_path - 语料库的位置
        Return:
        -------
            None
        Author:
        -------
            Ming Chen
        Modify:
        -------
            2019-11-27 13:42:50
        """
        with open(corpus_path, mode='r', encoding='utf-8') as fr:
            line = fr.readline()
            print('开始训练数据：')
            preline="\n"
            cnt=10000000
            
            while line:
                
                #print(line)
                if len(line) == 1:
                    preline="\n"
                else:
                    cur_char,cur_tag=line[0],line[2:-1]
                    self.B[self.tag2idx[cur_tag]][ord(cur_char)] += 1
                    if len(preline) == 1:
                        self.pi[self.tag2idx[cur_tag]] += 1
                    else:
                        pre_char, pre_tag = preline[0],preline[2:-1]
                        self.A[self.tag2idx[pre_tag]][self.tag2idx[cur_tag]] += 1
                    preline=line
                line = fr.readline()
                
                cnt-=1
                if cnt==0:
                    print('write 1000,0000 rows')
                    cnt=10000000
                    
        print('参数统计结果(A,PI)')
        print(f'A:{self.A}')
        print(f'PI:{self.pi}')
        
        self.pi[self.pi == 0] = self.epsilon  # 防止数据下溢,对数据进行对数归一化
        self.pi = np.log(self.pi) - np.log(np.sum(self.pi))
        self.A[self.A == 0] = self.epsilon
        self.A = np.log(self.A) - np.log(np.sum(self.A, axis=1, keepdims=True))
        self.B[self.B == 0] = self.epsilon
        self.B = np.log(self.B) - np.log(np.sum(self.B, axis=1, keepdims=True))
        
        print('训练参数结果(A,B,PI)')
        print(f'A:{self.A}')
        print(f'B:{self.B}')
        print(f'PI:{self.pi}')
        
        self.save_Param(savedir)
    
    def save_Param(self,savedir):
        if savedir==None:
            savedir='param/BIO/method1/'
        np.savetxt(f'{savedir}A.csv',self.A,delimiter=',')
        np.savetxt(f'{savedir}B.csv',self.B,delimiter=',')
        np.savetxt(f'{savedir}PI.csv',self.pi,delimiter=',')
        print(f'训练参数已保存在 {savedir}A.csv')
        print()
    
    def load_Param(self,loaddir='param/BIO/method1/'):
        self.A=np.genfromtxt(f'{loaddir}A.csv',delimiter=',')
        self.B=np.genfromtxt(f'{loaddir}B.csv',delimiter=',')
        self.pi=np.genfromtxt(f'{loaddir}PI.csv',delimiter=',')
        print(f'训练参数(A,B,PI)已从 {loaddir}A.csv 中读取成功')
        print()
    
    
    def viterbi(self, Obs):
        """
        函数说明： 使用viterbi算法进行解码

        Parameter：
        ----------
            Obs - 要解码的文本string
        Return:
        -------
            path - 最可能的隐状态路径
        Author:
        -------
            Ming Chen
        Modify:
        -------
            2019-11-27 16:52:42
        """
        T = len(Obs)
        delta = np.zeros((T, self.n_tag))  # shape: 观测文本数量*7
        psi = np.zeros((T, self.n_tag))  # shape: 观测文本数量*7
        delta[0] = self.pi[:] + self.B[:, ord(Obs[0])]  # 初始化
        for i in range(1, T):
            temp = delta[i - 1].reshape(self.n_tag, -1) + self.A  # 这里运用到了矩阵的广播算法
            delta[i] = np.max(temp, axis=0)
            delta[i] = delta[i, :] + self.B[:, ord(Obs[i])]
            psi[i] = np.argmax(temp, axis=0)
        path = np.zeros(T)
        path[T - 1] = np.argmax(delta[T - 1])
        for i in range(T - 2, -1, -1):  # 回溯
            path[i] = int(psi[i + 1][int(path[i + 1])])
        return path

    def predict(self, Obs):
        """
        函数说明： 将文本进行命名实体识别

        Parameter：
        ----------
            Obs - 要识别的文本
        Return:
        -------
            None
        Author:
        -------
            Ming Chen
        Modify:
        -------
            2019-11-27 20:53:23
        """
        T = len(Obs)
        path = self.viterbi(Obs)
        for i in range(T):
            print(Obs[i]+self.idx2tag[path[i]]+'_|', end='')
        print()
        print()

# BIO_Method 2

In [22]:
class HMM_BIO_method2:
    def __init__(self):
        # 标记-id
        self.tag2id = {'B-ORG': 0,
                       'I-ORG': 1,
                       'O': 2}
        # id-标记
        self.id2tag = dict(zip(self.tag2id.values(), self.tag2id.keys()))
        # 表示所有可能的标签个数N
        self.num_tag = len(self.tag2id)
        # 所有字符的Unicode编码个数 x16
        self.num_char = 65535
        # 转移概率矩阵,N*N
        self.A = np.zeros((self.num_tag, self.num_tag))
        # 发射概率矩阵,N*M
        self.B = np.zeros((self.num_tag, self.num_char))
        # 初始隐状态概率,N
        self.pi = np.zeros(self.num_tag)
        # 无穷小量
        self.epsilon = 1e-100

    def train(self, corpus_path , savedir=None):
        '''
        函数功能：通过数据训练得到A、B、pi
        :param corpus_path: 数据集文件路径
        :return: 无返回值
        '''
        with open(corpus_path, mode='r', encoding='utf-8') as f:
            # 读取训练数据
            lines = f.readlines()
        print('开始训练数据：')
        for i in tqdm(range(len(lines))):
            if len(lines[i]) == 1:
                # 空行，即只有一个换行符，跳过
                continue
            else:
                # split()的时候，多个空格当成一个空格
                cut_char, cut_tag = lines[i][0],lines[i][2:-1]
                # ord是python内置函数
                # ord(c)返回字符c对应的十进制整数
                self.B[self.tag2id[cut_tag]][ord(cut_char)] += 1
                if len(lines[i - 1]) == 1:
                    # 如果上一个数据是空格
                    # 即当前为一句话的开头
                    # 即初始状态
                    self.pi[self.tag2id[cut_tag]] += 1
                    continue
                pre_char, pre_tag = lines[i-1][0],lines[i-1][2:-1]
                self.A[self.tag2id[pre_tag]][self.tag2id[cut_tag]] += 1
                
        print('参数统计结果(A,PI)')
        print(f'A:{self.A}')
        print(f'PI:{self.pi}')
                
                
        
        # 为矩阵中所有是0的元素赋值为epsilon
        self.pi[self.pi == 0] = self.epsilon
        # 防止数据下溢,对数据进行对数归一化
        self.pi = np.log(self.pi) - np.log(np.sum(self.pi))
        self.A[self.A == 0] = self.epsilon
        # axis=1将每一行的元素相加，keepdims=True保持其二维性
        self.A = np.log(self.A) - np.log(np.sum(self.A, axis=1, keepdims=True))
        self.B[self.B == 0] = self.epsilon
        self.B = np.log(self.B) - np.log(np.sum(self.B, axis=1, keepdims=True))
        
        print('训练参数结果(A,B,PI)')
        print(f'A:{self.A}')
        print(f'B:{self.B}')
        print(f'PI:{self.pi}')
        print('训练完毕！')
        
        self.save_Param(savedir)
        
    def save_Param(self,savedir):
        if savedir==None:
            savedir='param/BIO/method2/'
        np.savetxt(f'{savedir}A.csv',self.A,delimiter=',')
        np.savetxt(f'{savedir}B.csv',self.B,delimiter=',')
        np.savetxt(f'{savedir}PI.csv',self.pi,delimiter=',')
        print(f'训练参数已保存在 {savedir}A.csv')
        print()
    
    def load_Param(self,loaddir='param/BIO/method2/'):
        self.A=np.genfromtxt(f'{loaddir}A.csv',delimiter=',')
        self.B=np.genfromtxt(f'{loaddir}B.csv',delimiter=',')
        self.pi=np.genfromtxt(f'{loaddir}PI.csv',delimiter=',')
        print(f'训练参数(A,B,PI)已从 {loaddir}A.csv 中读取成功')
        print()
    
    
    def viterbi(self, Obs):
        '''
        函数功能：使用viterbi算法进行解码
        :param Obs: 要解码的文本字符串
        :return: 最可能的隐状态路径
        '''
        # 获得观测序列的文本长度
        T = len(Obs)
        # T*N
        delta = np.zeros((T, self.num_tag))
        # T*N
        psi = np.zeros((T, self.num_tag))
        # ord是python内置函数
        # ord(c)返回字符c对应的十进制整数
        # 初始化
        delta[0] = self.pi[:] + self.B[:, ord(Obs[0])]
        # range（）左闭右开
        for i in range(1, T):
            # arr.reshape(4,-1) 将arr变成4行的格式，列数自动计算的(c=4, d=16/4=4)
            temp = delta[i - 1].reshape(self.num_tag, -1) + self.A
            # 按列取最大值
            delta[i] = np.max(temp, axis=0)
            # 得到delta值
            delta[i] = delta[i, :] + self.B[:, ord(Obs[i])]
            # 取出元素最大值对应的索引
            psi[i] = np.argmax(temp, axis=0)
        # 最优路径回溯
        path = np.zeros(T)
        path[T - 1] = np.argmax(delta[T - 1])
        for i in range(T - 2, -1, -1):
            path[i] = int(psi[i + 1][int(path[i + 1])])
        return path

    def predict(self, Obs):
        '''
        函数功能：对文本进行命名实体识别
        :param Obs: 要识别的文本
        :return: 无返回
        '''
        T = len(Obs)
        path = self.viterbi(Obs)
        for i in range(T):
            print(Obs[i] + self.id2tag[path[i]] + '_|', end='')
        print()
        print()



# BIO_Method 3

In [23]:
class HMM_BIO_method3:
    def __init__(self,filename='corpus/corpus_1.txt'):
        
        self.filename=filename
        # 初始化词典
        self.tag2id, self.id2tag  = {},{} # tag 表示实体标签，如 B-LOC E-LOC 等
        self.word2id, self.id2word = {},{} # word 表示中文字符

        # 根据数据建立词典
        for line in open("corpus/corpus_1.txt",encoding='utf-8'):
            if line == "\n":
                continue
            word, tag = line[0],line[2:-1]

            if word not in self.word2id:
                self.word2id[word] = len(self.word2id)
                self.id2word[len(self.id2word)] = word

            if tag not in self.tag2id:
                self.tag2id[tag] = len(self.tag2id)
                self.id2tag[len(self.id2tag)] = tag
        print(self.tag2id)

        # 初始化参数
        self.M = len(self.word2id)
        self.N = len(self.tag2id)

        self.pi = np.zeros(self.N)
        self.A = np.zeros((self.N,self.N))
        self.B = np.zeros((self.N,self.M))

        print(self.M)
        print(self.N)
    
    def train(self):
        # 统计参数
        pre_tag = ""
        for line in open(self.filename,encoding='utf-8'):
            if line == "\n": # 遇到空行跳过本次统计
                pre_tag = "" # 同时将 pre_tag 设为空
                continue

            items = [line[0],line[2:-1]]
            wordId, tagId = self.word2id[items[0]], self.tag2id[items[1].rstrip()]

            if pre_tag == "":
                self.pi[tagId] += 1
                self.B[tagId][wordId] += 1
            else:
                self.B[tagId][wordId] += 1
                self.A[self.tag2id[pre_tag]][tagId] += 1

            pre_tag = items[1].strip() # 为下个时刻记录本时刻的 pre_tag
        print('参数统计结果(A,pi)')
        print(f'A:{self.A}')
        print(f'pi:{self.pi}')

        #转化为概率的形式
        self.pi = self.pi/sum(self.pi)
        for i in range(self.N):
            self.A[i] /= sum(self.A[i])
            self.B[i] /= sum(self.B[i])
        
        print('训练参数结果(A,B,PI)')
        print(f'A:{self.A}')
        print(f'B:{self.B}')
        print(f'PI:{self.pi}')
        

    #概率连乘的前面加log变为加法
    def log(self,v):
        if v==0:
            return np.log(v+0.00001)
        return np.log(v)

    #维特比算法
    def predict(self,x):
        y = x
        x = [self.word2id[word] for word in x]
        T = len(x)

        dp = np.zeros((T,self.N))  # T 是序列长度，self.N 是状态总数
        ptr = np.zeros((T,self.N),dtype=int) # 存放下标

        for j in range(self.N):
            dp[0][j] = self.log(self.pi[j]) + self.log(self.B[j][x[0]])   # t = 1 时刻的得分单独计算

        # 从第二个时刻开始由上至下、由左至右地更新DP数组
        for i in range(1, T):
            for j in range(self.N):
                dp[i][j] = -99999
                for k in range(self.N):
                    score = dp[i-1][k] + self.log(self.A[k][j]) + self.log(self.B[j][x[i]])
                    if score > dp[i][j]:   # 如果得分高于先前，更新DP数组
                        dp[i][j] = score
                        ptr[i][j] = k  # 记录路径

        best_seq = [0]*T
        best_seq[T-1] = np.argmax(dp[T-1]) # 取最后时刻的DP数组中最大值的下标

        for i in range(T-2, -1, -1):# 从 T-1 遍历到 0 时刻
            best_seq[i] = ptr[i+1][best_seq[i+1]]

        ans=""
        for i in range(len(best_seq)):
            ans+=y[i]+self.id2tag[best_seq[i]]+'_|'
        print(ans)

# BIOES_Main

In [48]:
#隐马尔可夫模型
#面向命名实体识别(组织机构)问题
class HMM_BIOES_main:
    def __init__(self,A=None,B=None,PI=None):
        #param:
        # (A,B,PI) 状态转移矩阵 发射矩阵 初始状态矩阵
        self.print=True
        
        
        self.N = 4     # 状态集合有多少元素(状态数BIO)
        self.M = 65535 # 观测集合有多少元素(汉字)
        
        #数字化 状态集合Q 观测集合V 
        self.status_dict={ # {'盒子1':0，'盒子2':1，'盒子3':2}
                        'B-ORG': 0,
                        'I-ORG': 1,
                        'E-ORG': 2,
                        'O': 3}
        self.status=list(self.status_dict.keys())          
        #self.observe_dict={}# {'红':0，'白':1}
        self.Q=np.arange(0,self.N) #状态集合 [0,1,2]
        self.V=np.arange(0,self.M) #观测集合 [0,1]
        
        #初始化 (A,B,PI)
        self.A=A # 状态转移矩阵
        self.B=B # 发射矩阵
        self.PI=PI # 初始状态概率
        
        
        if self.print:
            print('状态集合status',self.status_dict)
            #print('观测集合observe',self.observe_dict)
            print('状态集合Q',self.Q)
            print('观测集合V',self.V)
            print()
        

    
    #训练模型(A,B,PI)
    #data (2,N)
    def train(self,filename,savedir=None):
        #利用数据data(状态序列+观测序列) 计算参数(A,B,PI) 直接使用统计方法 统计A,B,PI
        
        A=np.zeros((self.N,self.N)) #(N,N)
        B=np.zeros((self.N,self.M)) #(N,M)
        PI=np.zeros(self.N) #(N,)
        
        
        with open(filename,'r',encoding='utf-8') as fr:
            mult=10000000
            cnt=0
            
            line=fr.readline()
            pres=-1
            while line:
        
                mult-=1
                if mult==0:
                    cnt+=1
                    print(f'load {10000000}*{cnt} rows')
                    mult=10000000
                    
                
                
                s,v=line.split(',')
                s=int(s)
                v=int(v[:-1])
                #print(line,s,v)
                #当前为分隔符
                if s==-1 or v>=self.M:
                    pres=-1
                    line=fr.readline()
                    continue
                if pres!=-1:
                    #s 当前状态
                    #pres 上一次状态
                    #v 当前观测

                    A[pres,s]+=1
                    B[s,v]+=1
                    
                #上一个为分隔符
                else:
                    PI[s]+=1
                    B[s,v]+=1
                pres=s
                line=fr.readline()
        
        self.A=A
        self.B=B
        self.PI=PI
        print('参数统计结果(A,PI)')
        print(f'A:{self.A}')
        print(f'PI:{self.PI}')
        
        
        self.normalize_param()
        print('训练参数结果(A,B,PI)')
        print(f'A:{self.A}')
        print(f'B:{self.B}')
        print(f'PI:{self.PI}')
        
        
        self.save_Param(savedir)
        
        
    
    def normalize_param(self):
        epsilon=1e-8
        
        
        self.PI[self.PI == 0] =epsilon  # 防止数据下溢,对数据进行对数归一化
        self.PI = np.log(self.PI) - np.log(np.sum(self.PI))

        self.A[self.A == 0] = epsilon
        self.A = np.log(self.A) - np.log(np.sum(self.A, axis=1, keepdims=True))

        self.B[self.B == 0] = epsilon
        self.B = np.log(self.B) - np.log(np.sum(self.B, axis=1, keepdims=True))
        
        #将(A,B,PI)归一化 (直接归一化)
        #self.A=self.A/np.sum(self.A,axis=1,keepdims=True)
        #self.B=self.B/np.sum(self.B,axis=1,keepdims=True)
        #self.PI=self.PI/np.sum(self.PI)

    def save_Param(self,savedir):
        if savedir==None:
            savedir='param/BIOES/main/'
        np.savetxt(f'{savedir}A.csv',self.A,delimiter=',')
        np.savetxt(f'{savedir}B.csv',self.B,delimiter=',')
        np.savetxt(f'{savedir}PI.csv',self.PI,delimiter=',')
        print(f'训练参数已保存在 {savedir}A.csv')
        print()
    
    def load_Param(self,loaddir='param/BIOES/main/'):
        self.A=np.genfromtxt(f'{loaddir}A.csv',delimiter=',')
        self.B=np.genfromtxt(f'{loaddir}B.csv',delimiter=',')
        self.PI=np.genfromtxt(f'{loaddir}PI.csv',delimiter=',')
        print(f'训练参数(A,B,PI)已从 {loaddir}A.csv 中读取成功')
        print()
    
    def viterbi_t(self,o):
        #维特比算法(动态规划)
        #已知(A,B,PI) 和一观测序列o 求解其状态序列的概率最大解
        #param:
        # o 观测序列 [0,1,1,1,1,0] shape(n,)
        #return:
        # ret 状态序列 shape(n,)
        #solution:
        # delta计算从前一个的所有状态 到当前状态的概率最大值 psi记录下来概率最大值的前一个状态
        # 从delta的最后取最大值，利用psi向前回溯即可找到最大概率序列
        
        n=o.shape[0]
        
        delta=np.zeros((self.N,n)) #delta[:,i] 为到达该观测序列的 每个状态的最大概率值 shape(状态个数,序列长度)
        psi=np.zeros((self.N,n),dtype=np.int32) #psi[:,i] 为到达该观测序列的 最大概率值的 上一个状态为哪个
        A=self.A
        B=self.B
        PI=self.PI
        
        #* -> + (因为log)
        delta[:,0]=PI+B[:,o[0]]
        #psi[:,0]=np.argmax(delta[:,0])
        for i in range(1,n):
            temp=delta[:,i-1]+A.T
            psi[:,i]=np.argmax(temp,axis=1)
            delta[:,i]=np.max(temp,axis=1)+B[:,o[i]]
        
        ret=np.zeros(n,dtype=np.int32)
        ret[-1]=np.argmax(delta[:,-1])

        for i in range(n-2,-1,-1):
            ret[i]=psi[ret[i+1]][i+1]
        
        
        
        return ret
    
    
    
    def calc_acc(self,label,predict):
        #计算标签预测的准确率
        #param:
        # a 源标签
        # b 目的标签
        return np.mean(np.equal(label,predict))
    
    
    
    def predict(self,s:str):
        #预测一段话的状态序列
        #param
        # s 一段中文
        #print
        # 中文加状态的文字 林B-LOC_|徽I-LOC_|因_O_|
        
        ans=""
        s=list(s)
        o=np.array([ord(ch) for ch in s])
        #打印观测序列
        #print(o)
        statusList=self.viterbi_t(o)
        for i in range(len(s)):
            ans+=s[i]+str(self.status[statusList[i]])+'_|'
        #print(statusList)
        print(ans)
        

# 所有模型训练

In [24]:
#BIO模型main训练
hmm_bio_main=HMM_BIO_main()
hmm_bio_main.train('corpus/BIO_train_pretreatment_1.csv')

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'O': 2}
状态集合Q [0 1 2]
观测集合V [    0     1     2 ... 65532 65533 65534]

参数统计结果(A,PI)
A:[[  0.  17.   0.]
 [  0. 122.  10.]
 [ 10.   0. 664.]]
PI:[ 7.  0. 88.]
训练参数结果(A,B,PI)
A:[[-2.12538941e+01 -1.17647048e-09 -2.12538941e+01]
 [-2.33034827e+01 -7.87808779e-02 -2.58021683e+00]
 [-4.21064502e+00 -2.49339109e+01 -1.49479615e-02]]
B:[[-21.25393263 -21.25393263 -21.25393263 ... -21.25393263 -21.25393263
  -21.25393263]
 [-23.35515939 -23.35515939 -23.35515939 ... -23.35515939 -23.35515939
  -23.35515939]
 [-25.05662816 -25.05662816 -25.05662816 ... -25.05662816 -25.05662816
  -25.05662816]]
PI:[ -2.60796674 -22.97455764  -0.07654008]
训练参数已保存在 param/BIO/main/A.csv



In [25]:
#BIO模型1训练
hmm_bio_method1=HMM_BIO_method1()
hmm_bio_method1.train('corpus/corpus_1.txt')

开始训练数据：
参数统计结果(A,PI)
A:[[  0.  17.   0.]
 [  0. 122.  10.]
 [ 10.   0. 664.]]
PI:[ 7.  0. 88.]
训练参数结果(A,B,PI)
A:[[-2.33091723e+02  0.00000000e+00 -2.33091723e+02]
 [-2.35141311e+02 -7.87808779e-02 -2.58021683e+00]
 [-4.21064502e+00 -2.36771739e+02 -1.49479614e-02]]
B:[[-233.09172264 -233.09172264 -233.09172264 ... -233.09172264
  -233.09172264 -233.09172264]
 [-235.19298323 -235.19298323 -235.19298323 ... -235.19298323
  -235.19298323 -235.19298323]
 [-236.89445586 -236.89445586 -236.89445586 ... -236.89445586
  -236.89445586 -236.89445586]]
PI:[-2.60796674e+00 -2.34812386e+02 -7.65400771e-02]
训练参数已保存在 param/BIO/method1/A.csv



In [47]:
#BIO模型2训练
hmm_bio_method2=HMM_BIO_method2()
hmm_bio_method2.train('corpus/corpus_1.txt')

开始训练数据：


100%|██████████| 1015/1015 [00:00<00:00, 516214.21it/s]

参数统计结果(A,PI)
A:[[  0.  17.   0.]
 [  0. 122.  10.]
 [ 10.   0. 664.]]
PI:[ 7.  0. 88.]
训练参数结果(A,B,PI)
A:[[-2.33091723e+02  0.00000000e+00 -2.33091723e+02]
 [-2.35141311e+02 -7.87808779e-02 -2.58021683e+00]
 [-4.21064502e+00 -2.36771739e+02 -1.49479614e-02]]
B:[[-233.09172264 -233.09172264 -233.09172264 ... -233.09172264
  -233.09172264 -233.09172264]
 [-235.19298323 -235.19298323 -235.19298323 ... -235.19298323
  -235.19298323 -235.19298323]
 [-236.89445586 -236.89445586 -236.89445586 ... -236.89445586
  -236.89445586 -236.89445586]]
PI:[-2.60796674e+00 -2.34812386e+02 -7.65400771e-02]
训练完毕！
训练参数已保存在 param/BIO/method2/A.csv



In [46]:
#BIO模型3训练
hmm_bio_method3=HMM_BIO_method3('corpus/corpus_1.txt')
hmm_bio_method3.train()

{'O': 0, 'B-ORG': 1, 'I-ORG': 2}
261
3
参数统计结果(A,pi)
A:[[664.  10.   0.]
 [  0.   0.  17.]
 [ 10.   0. 122.]]
pi:[88.  7.  0.]
训练参数结果(A,B,PI)
A:[[0.9851632  0.0148368  0.        ]
 [0.         0.         1.        ]
 [0.07575758 0.         0.92424242]]
B:[[0.00262467 0.0144357  0.00656168 0.00918635 0.00131234 0.00131234
  0.00262467 0.01049869 0.03149606 0.01312336 0.03149606 0.0183727
  0.03937008 0.00787402 0.05249344 0.00656168 0.00131234 0.0144357
  0.01574803 0.00131234 0.00262467 0.00131234 0.00131234 0.00131234
  0.00656168 0.00131234 0.0144357  0.00656168 0.00131234 0.00131234
  0.00131234 0.         0.00918635 0.00656168 0.00787402 0.00918635
  0.00787402 0.00787402 0.01049869 0.00787402 0.00787402 0.00131234
  0.00131234 0.00131234 0.00787402 0.00524934 0.00524934 0.01706037
  0.01706037 0.00524934 0.00524934 0.00656168 0.00393701 0.00393701
  0.00262467 0.00787402 0.00262467 0.00262467 0.00393701 0.00393701
  0.00131234 0.00131234 0.00262467 0.         0.         0.
  0.    

In [50]:
#BIOES模型main训练
hmm_bioes_main=HMM_BIOES_main()
hmm_bioes_main.train('corpus/BIO_train_pretreatment_1.csv')

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'E-ORG': 2, 'O': 3}
状态集合Q [0 1 2 3]
观测集合V [    0     1     2 ... 65532 65533 65534]

参数统计结果(A,PI)
A:[[  0.  17.   0.   0.]
 [  0. 122.  10.   0.]
 [ 10.   0. 664.   0.]
 [  0.   0.   0.   0.]]
PI:[ 7.  0. 88.  0.]
训练参数结果(A,B,PI)
A:[[-2.12538941e+01 -1.76470616e-09 -2.12538941e+01 -2.12538941e+01]
 [-2.33034827e+01 -7.87808780e-02 -2.58021683e+00 -2.33034827e+01]
 [-4.21064502e+00 -2.49339109e+01 -1.49479615e-02 -2.49339109e+01]
 [-1.38629436e+00 -1.38629436e+00 -1.38629436e+00 -1.38629436e+00]]
B:[[-21.25393263 -21.25393263 -21.25393263 ... -21.25393263 -21.25393263
  -21.25393263]
 [-23.35515939 -23.35515939 -23.35515939 ... -23.35515939 -23.35515939
  -23.35515939]
 [-25.05662816 -25.05662816 -25.05662816 ... -25.05662816 -25.05662816
  -25.05662816]
 [-11.09033963 -11.09033963 -11.09033963 ... -11.09033963 -11.09033963
  -11.09033963]]
PI:[ -2.60796674 -22.97455764  -0.07654008 -22.97455764]
训练参数已保存在 param/BIOES/mainA.csv



# 所有模型验证

## corpus_1.txt (7KB) 使用BIO四个模型

In [28]:
sentences=['龙川县博物馆馆藏文物','林徽因什么理由拒绝了徐志摩而选择梁思成变为终身伴侣?谢娜为李浩菲澄清网络谣言，之后她的两个行为给自己加分','王心雷编写了一个代码',
      '王康在哈尔滨出门打电动被张舒帆当场逮捕','北京抖音信息服务有限公司今天出台了一个新的政策','中国招标网发布了一个新的招标',
       '我真的好饿','中标人:遂宁高新区蜜感内衣店','中国是个好地方','宁夏最高气温20度','恭喜安居区安居镇有家超市昨天开业了',
           '家乐福是个好地方','家乐福','家乐福超市']

In [29]:
for sentence in sentences:
    hmm_bio_main.predict(sentence)
    print()
    
    hmm_bio_method1.predict(sentence)
    print()
    
    hmm_bio_method2.predict(sentence)
    print()
    
    print('---------------------------------------------------------------------')
    

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏O_|文O_|物O_|

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏O_|文O_|物O_|


龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏O_|文O_|物O_|


---------------------------------------------------------------------
林B-ORG_|徽I-ORG_|因I-ORG_|什I-ORG_|么I-ORG_|理I-ORG_|由I-ORG_|拒I-ORG_|绝I-ORG_|了I-ORG_|徐I-ORG_|志O_|摩O_|而O_|选O_|择B-ORG_|梁I-ORG_|思I-ORG_|成O_|变B-ORG_|为I-ORG_|终I-ORG_|身I-ORG_|伴I-ORG_|侣I-ORG_|?I-ORG_|谢I-ORG_|娜I-ORG_|为I-ORG_|李I-ORG_|浩I-ORG_|菲I-ORG_|澄I-ORG_|清I-ORG_|网I-ORG_|络I-ORG_|谣I-ORG_|言I-ORG_|，I-ORG_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

林B-ORG_|徽I-ORG_|因I-ORG_|什I-ORG_|么I-ORG_|理I-ORG_|由I-ORG_|拒I-ORG_|绝I-ORG_|了I-ORG_|徐I-ORG_|志O_|摩O_|而O_|选O_|择B-ORG_|梁I-ORG_|思I-ORG_|成O_|变B-ORG_|为I-ORG_|终I-ORG_|身I-ORG_|伴I-ORG_|侣I-ORG_|?I-ORG_|谢I-ORG_|娜I-ORG_|为I-ORG_|李I-ORG_|浩I-ORG_|菲I-ORG_|澄I-ORG_|清I-ORG_|网I-ORG_|络I-ORG_|谣I-ORG_|言I-ORG_|，I-ORG_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|


林B-ORG_|徽I-ORG_|因I-ORG_|什

In [30]:
#验证模型3
words=["中国",'龙川县博物馆馆藏文物']
for word in words:
    hmm_bio_main.predict(word)
    print()
    
    hmm_bio_method1.predict(word)
    print()
    
    hmm_bio_method2.predict(word)
    print()
    
    hmm_bio_method3.predict(word)
    print()

中O_|国B-ORG_|

中O_|国B-ORG_|


中O_|国B-ORG_|


中O_|国B-ORG_|

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏O_|文O_|物O_|

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏O_|文O_|物O_|


龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏O_|文O_|物O_|


龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏O_|文O_|物O_|



## corpus_1.txt (7KB) corpus_part0.txt (413MB) corpus.txt(9GB) 对比 使用bio_main模型

In [35]:
hmm_bio_main1=HMM_BIO_main()
hmm_bio_main1.load_Param('param/BIO/corpus_7K/')

hmm_bio_main2=HMM_BIO_main()
hmm_bio_main2.load_Param('param/BIO/corpus_400M/')

hmm_bio_main3=HMM_BIO_main()
hmm_bio_main3.load_Param('param/BIO/corpus_9G/')

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'O': 2}
状态集合Q [0 1 2]
观测集合V [    0     1     2 ... 65532 65533 65534]

训练参数(A,B,PI)已从 param/BIO/corpus_7K/A.csv 中读取成功

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'O': 2}
状态集合Q [0 1 2]
观测集合V [    0     1     2 ... 65532 65533 65534]

训练参数(A,B,PI)已从 param/BIO/corpus_400M/A.csv 中读取成功

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'O': 2}
状态集合Q [0 1 2]
观测集合V [    0     1     2 ... 65532 65533 65534]

训练参数(A,B,PI)已从 param/BIO/corpus_9G/A.csv 中读取成功



In [36]:
for sentence in sentences:
    hmm_bio_main1.predict(sentence)#7KB
    print()
    
    hmm_bio_main2.predict(sentence)#400MB
    print()
    
    hmm_bio_main3.predict(sentence)#9G
    print()
    
    print('---------------------------------------------------------------------')

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏O_|文O_|物O_|

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏I-ORG_|文I-ORG_|物I-ORG_|

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏I-ORG_|文I-ORG_|物I-ORG_|

---------------------------------------------------------------------
林B-ORG_|徽I-ORG_|因I-ORG_|什I-ORG_|么I-ORG_|理I-ORG_|由I-ORG_|拒I-ORG_|绝I-ORG_|了I-ORG_|徐I-ORG_|志O_|摩O_|而O_|选O_|择B-ORG_|梁I-ORG_|思I-ORG_|成O_|变B-ORG_|为I-ORG_|终I-ORG_|身I-ORG_|伴I-ORG_|侣I-ORG_|?I-ORG_|谢I-ORG_|娜I-ORG_|为I-ORG_|李I-ORG_|浩I-ORG_|菲I-ORG_|澄I-ORG_|清I-ORG_|网I-ORG_|络I-ORG_|谣I-ORG_|言I-ORG_|，I-ORG_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

林O_|徽O_|因O_|什O_|么O_|理O_|由O_|拒O_|绝O_|了O_|徐O_|志O_|摩O_|而O_|选O_|择O_|梁O_|思O_|成O_|变O_|为O_|终O_|身O_|伴O_|侣O_|?O_|谢O_|娜O_|为O_|李O_|浩O_|菲O_|澄O_|清O_|网O_|络O_|谣O_|言O_|，O_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

林B-ORG_|徽I-ORG_|因I-ORG_|什I-ORG_|么I-ORG_|理I-ORG_|由O_|拒O_|绝O_|了O_|徐O_|志O_|摩O_|而O_|选O_|择O_|梁O_|思O_|成O_|变O_|为O_|终O_|身O_|伴O_|侣O_|?O_|谢O_|娜O_|为O_|

## BIO BIOES模型 corpus_1.txt (7KB) corpus_part0.txt (413MB) corpus.txt(9GB) 对比

In [37]:
#hmm_bio加载
hmm_bio_main1=HMM_BIO_main()
hmm_bio_main1.load_Param('param/BIO/corpus_7K/')

hmm_bio_main2=HMM_BIO_main()
hmm_bio_main2.load_Param('param/BIO/corpus_400M/')

hmm_bio_main3=HMM_BIO_main()
hmm_bio_main3.load_Param('param/BIO/corpus_9G/')

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'O': 2}
状态集合Q [0 1 2]
观测集合V [    0     1     2 ... 65532 65533 65534]

训练参数(A,B,PI)已从 param/BIO/corpus_7K/A.csv 中读取成功

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'O': 2}
状态集合Q [0 1 2]
观测集合V [    0     1     2 ... 65532 65533 65534]

训练参数(A,B,PI)已从 param/BIO/corpus_400M/A.csv 中读取成功

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'O': 2}
状态集合Q [0 1 2]
观测集合V [    0     1     2 ... 65532 65533 65534]

训练参数(A,B,PI)已从 param/BIO/corpus_9G/A.csv 中读取成功



In [51]:
#hmm_bio加载
hmm_bioes_main1=HMM_BIOES_main()
hmm_bioes_main1.load_Param('param/BIOES/corpus_7K/')

hmm_bioes_main2=HMM_BIOES_main()
hmm_bioes_main2.load_Param('param/BIOES/corpus_400M/')

hmm_bioes_main3=HMM_BIOES_main()
hmm_bioes_main3.load_Param('param/BIOES/corpus_9G/')

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'E-ORG': 2, 'O': 3}
状态集合Q [0 1 2 3]
观测集合V [    0     1     2 ... 65532 65533 65534]

训练参数(A,B,PI)已从 param/BIOES/corpus_7K/A.csv 中读取成功

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'E-ORG': 2, 'O': 3}
状态集合Q [0 1 2 3]
观测集合V [    0     1     2 ... 65532 65533 65534]

训练参数(A,B,PI)已从 param/BIOES/corpus_400M/A.csv 中读取成功

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'E-ORG': 2, 'O': 3}
状态集合Q [0 1 2 3]
观测集合V [    0     1     2 ... 65532 65533 65534]

训练参数(A,B,PI)已从 param/BIOES/corpus_9G/A.csv 中读取成功



### 7KB

In [52]:
for sentence in sentences:
    hmm_bio_main1.predict(sentence)#7KB
    print()
    
    hmm_bioes_main1.predict(sentence)#7KB
    print()
    
    print('---------------------------------------------------------------------')

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏O_|文O_|物O_|

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆E-ORG_|馆O_|藏O_|文O_|物O_|

---------------------------------------------------------------------
林B-ORG_|徽I-ORG_|因I-ORG_|什I-ORG_|么I-ORG_|理I-ORG_|由I-ORG_|拒I-ORG_|绝I-ORG_|了I-ORG_|徐I-ORG_|志O_|摩O_|而O_|选O_|择B-ORG_|梁I-ORG_|思I-ORG_|成O_|变B-ORG_|为I-ORG_|终I-ORG_|身I-ORG_|伴I-ORG_|侣I-ORG_|?I-ORG_|谢I-ORG_|娜I-ORG_|为I-ORG_|李I-ORG_|浩I-ORG_|菲I-ORG_|澄I-ORG_|清I-ORG_|网I-ORG_|络I-ORG_|谣I-ORG_|言I-ORG_|，I-ORG_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

林B-ORG_|徽I-ORG_|因I-ORG_|什I-ORG_|么I-ORG_|理I-ORG_|由I-ORG_|拒I-ORG_|绝I-ORG_|了I-ORG_|徐E-ORG_|志O_|摩O_|而O_|选O_|择B-ORG_|梁I-ORG_|思E-ORG_|成O_|变B-ORG_|为I-ORG_|终I-ORG_|身I-ORG_|伴I-ORG_|侣I-ORG_|?I-ORG_|谢I-ORG_|娜I-ORG_|为I-ORG_|李I-ORG_|浩I-ORG_|菲I-ORG_|澄I-ORG_|清I-ORG_|网I-ORG_|络I-ORG_|谣I-ORG_|言I-ORG_|，E-ORG_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

---------------------------------------------------------------------
王B-ORG_|心I-ORG_|雷I-ORG_|编O_|写O_|

### 413MB

In [53]:
for sentence in sentences:
    hmm_bio_main2.predict(sentence)#400MB
    print()
    
    hmm_bioes_main2.predict(sentence)#400MB
    print()
    
    print('---------------------------------------------------------------------')

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏I-ORG_|文I-ORG_|物I-ORG_|

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏I-ORG_|文I-ORG_|物I-ORG_|

---------------------------------------------------------------------
林O_|徽O_|因O_|什O_|么O_|理O_|由O_|拒O_|绝O_|了O_|徐O_|志O_|摩O_|而O_|选O_|择O_|梁O_|思O_|成O_|变O_|为O_|终O_|身O_|伴O_|侣O_|?O_|谢O_|娜O_|为O_|李O_|浩O_|菲O_|澄O_|清O_|网O_|络O_|谣O_|言O_|，O_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

林O_|徽O_|因O_|什O_|么O_|理O_|由O_|拒O_|绝O_|了O_|徐O_|志O_|摩O_|而O_|选O_|择O_|梁O_|思O_|成O_|变O_|为O_|终O_|身O_|伴O_|侣O_|?O_|谢O_|娜O_|为O_|李O_|浩O_|菲O_|澄O_|清O_|网O_|络O_|谣O_|言O_|，O_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

---------------------------------------------------------------------
王O_|心O_|雷O_|编O_|写O_|了O_|一O_|个O_|代O_|码O_|

王O_|心O_|雷O_|编O_|写O_|了O_|一O_|个O_|代O_|码O_|

---------------------------------------------------------------------
王O_|康O_|在O_|哈O_|尔O_|滨O_|出O_|门O_|打O_|电O_|动O_|被O_|张O_|舒O_|帆O_|当O_|场O_|逮O_|捕O_|

王O_|康O_|在O_|哈O_|尔O_|滨O_|出O_|门O_|打O_|电O_|动O_|

### 9GB

In [54]:
for sentence in sentences:
    hmm_bio_main3.predict(sentence)#9GB
    print()
    
    hmm_bioes_main3.predict(sentence)#9GB
    print()
    
    print('---------------------------------------------------------------------')

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏I-ORG_|文I-ORG_|物I-ORG_|

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏I-ORG_|文I-ORG_|物I-ORG_|

---------------------------------------------------------------------
林B-ORG_|徽I-ORG_|因I-ORG_|什I-ORG_|么I-ORG_|理I-ORG_|由O_|拒O_|绝O_|了O_|徐O_|志O_|摩O_|而O_|选O_|择O_|梁O_|思O_|成O_|变O_|为O_|终O_|身O_|伴O_|侣O_|?O_|谢O_|娜O_|为O_|李O_|浩O_|菲O_|澄O_|清O_|网O_|络O_|谣O_|言O_|，O_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

林O_|徽O_|因O_|什O_|么O_|理O_|由O_|拒O_|绝O_|了O_|徐O_|志O_|摩O_|而O_|选O_|择O_|梁O_|思O_|成O_|变O_|为O_|终O_|身O_|伴O_|侣O_|?O_|谢O_|娜O_|为O_|李B-ORG_|浩I-ORG_|菲I-ORG_|澄I-ORG_|清I-ORG_|网I-ORG_|络I-ORG_|谣I-ORG_|言I-ORG_|，E-ORG_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

---------------------------------------------------------------------
王O_|心O_|雷O_|编O_|写O_|了O_|一O_|个O_|代O_|码O_|

王O_|心O_|雷O_|编O_|写O_|了O_|一O_|个O_|代O_|码O_|

---------------------------------------------------------------------
王O_|康O_|在O_|哈B-ORG_|尔I-ORG_|滨I-ORG_|出O_|门O_|打O_|电O_|动O_|被O